In [11]:
import re

def extract_patterns(file_path, pattern): # Check if the file exists

    try:
        # Open the file and read it line by line
        with open(file_path, 'r') as file:
            matches = [line.strip() for line in file if re.search(pattern, line)] ## Search for the pattern in each line
            return matches if matches else "No matches found."
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except Exception as e:
        return f"Error: Something went wrong. Details: {e}"

file_path = r'C:\Users\User\Desktop\Assessment\Assessment 18/log.txt'
pattern = r'ERROR:'  # Example pattern to match error lines
result = extract_patterns(file_path, pattern)
print(result)

['ERROR: Failed to connect to database', 'ERROR: Invalid user input']


In [28]:
def process_chunk(chunk):
    # Example: Print the first line of the chunk
    if chunk:
        print(f"First line of chunk: {chunk[0]}")

def write_checkpoint(chunk_count):
    with open("checkpoint.txt", "w") as f:
        f.write(f"Last processed chunk: {chunk_count}")

def process_csv_in_chunks(file_path, chunk_size=1000):
    with open(file_path, 'r') as f:
        chunk_count = 0
        while True:
            try:
                chunk = []
                for _ in range(chunk_size):
                    line = f.readline()
                    if not line:
                        break
                    chunk.append(line)

                if not chunk:
                    break  # End of file

                # Process the chunk (e.g., data transformation, analysis)
                process_chunk(chunk)

                # Write checkpoint (e.g., update a log file)
                write_checkpoint(chunk_count)

                chunk_count += 1

            except Exception as e:
                print(f"Error processing chunk {chunk_count}: {e}")
                # Handle the error (e.g., log, retry, skip chunk) 

if __name__ == "__main__":
    process_csv_in_chunks(r"C:\Users\User\Desktop\Assessment\Assessment 18\file.csv")

First line of chunk: Name,Age,Date,Amount,Category,City



In [27]:
import pandas as pd
import os

def process_financial_file(file_path, expected_format="csv", max_size_mb=5):
    """Reads financial data safely, ensuring valid format and size."""
    
    if not os.path.exists(file_path):
        return "Error: File not found."
    
    if os.path.getsize(file_path) > max_size_mb * 1024 * 1024:
        return f"Error: File too large (max {max_size_mb}MB)."

    try:
        if expected_format == "csv":
            df = pd.read_csv(file_path)
        elif expected_format == "excel":
            df = pd.read_excel(file_path)
        elif expected_format == "json":
            df = pd.read_json(file_path)
        else:
            return "Error: Unsupported file format."

        if not {"Date", "Amount", "Category"}.issubset(df.columns):
            return "Error: Missing required columns."

        return df  # Successfully loaded data

    except Exception as e:
        return f"Error: Could not read file ({e})"

# Example Usage
result = process_financial_file(r"C:\Users\User\Desktop\Assessment\Assessment 18\file.csv")

print(result if isinstance(result, pd.DataFrame) else result)  # Show data or error


    Name  Age      Date  Amount  Category City
0   john   24  7/4/2024  100.50       NaN    H
1   jane   33  7/5/2024  130.50       NaN    G
2  danny   45  7/6/2024   50.25       NaN    C
3   hims   55  7/7/2024   70.25       NaN    A


In [2]:
def guess_number(secret_number, max_attempts): #function. Takes the secret_number and max_attempts as input.

  for attempt in range(1, max_attempts + 1): #Uses a for loop to control the number of attempts.
    guess = int(input(f"Guess {attempt}/{max_attempts}: ")) #Gets the player's guess.
    if guess == secret_number:
      print("You win!") #If correct, prints "You win!" and returns True.
      return True 
    print("Try again.") #If incorrect, prints "Try again." and continues to the next attempt.
  print("You lose!") #If the loop finishes without a correct guess, it prints "You lose!" and returns False.
  return False 

# Sets the secret_number and max_tries.
secret_number = 5 
max_tries = 3
if guess_number(secret_number, max_tries): #Calls the guess_number function.
  print("Congratulations!") #Prints a winning or losing message based on the function's return value.

Guess 1/3:  34


Try again.


Guess 2/3:  5


You win!
Congratulations!


In [23]:
import logging

def setup_logging(log_level='INFO', log_file=None): #setup_logging function. Takes log_level and log_file as optional arguments
  #Uses logging.basicConfig to configure logging
    logging.basicConfig(
      level=log_level, #level: Sets the minimum logging level.
      format='%(asctime)s - %(levelname)s - %(message)s', #format: Defines the format for log messages.
      filename=log_file #filename: If provided, logs messages to the specified file.
  )

setup_logging(log_level='DEBUG', log_file='my_app.log') #Calls setup_logging with desired settings.

#Uses logging.debug(), logging.info(), etc. to log messages.

# Use the logger
logging.debug("This is a debug message.")
logging.info("This is an info message.")

In [21]:
import logging  #Use logging module
import sqlite3

def setup_logger(log_to_console=True, log_to_file=None, log_to_db=None): #Configures a logger with console, file, or database logging.
    logger = logging.getLogger("AppLogger")
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    if log_to_console:
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)

    if log_to_file:
        file_handler = logging.FileHandler(log_to_file)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)

    if log_to_db:
        setup_db(log_to_db)
        logger.db_path = log_to_db  # Store DB path in logger for later use

    return logger

def setup_db(db_path): #Creates a logs table in the SQLite database if it doesn't exist.
    conn = sqlite3.connect(db_path)
    conn.execute('CREATE TABLE IF NOT EXISTS logs (timestamp TEXT, level TEXT, message TEXT)')
    conn.commit()
    conn.close()

def log_to_database(logger, level, message): #Logs a message to the database if enabled.
    if hasattr(logger, 'db_path'):
        conn = sqlite3.connect(logger.db_path)
        conn.execute('INSERT INTO logs VALUES (datetime("now"), ?, ?)', (level.upper(), message))
        conn.commit()
        conn.close()

def log_message(logger, level, message): #Logs a message to the selected outputs.
    getattr(logger, level.lower())(message)  # Log to console/file
    log_to_database(logger, level, message)  # Log to DB if enabled

logger = setup_logger(log_to_console=True, log_to_file="my_app.log", log_to_db="logs.db")
log_message(logger, "info", "Application started.")
log_message(logger, "error", "An error occurred.")


2025-01-29 19:07:55,708 - INFO - Application started.
2025-01-29 19:07:55,708 - INFO - Application started.
2025-01-29 19:07:55,708 - INFO - Application started.
2025-01-29 19:07:55,708 - INFO - Application started.
2025-01-29 19:07:55,708 - INFO - Application started.
2025-01-29 19:07:55,708 - INFO - Application started.
2025-01-29 19:07:55,725 - ERROR - An error occurred.
2025-01-29 19:07:55,725 - ERROR - An error occurred.
2025-01-29 19:07:55,725 - ERROR - An error occurred.
2025-01-29 19:07:55,725 - ERROR - An error occurred.
2025-01-29 19:07:55,725 - ERROR - An error occurred.
2025-01-29 19:07:55,725 - ERROR - An error occurred.


In [27]:
import re

def extract_hashtags(text): #extract_hashtags function
  hashtags = re.findall(r"#\w+", text) 
    # r"#\w+": This is the regular expression
    #  # : Matches the literal "#" character.
    # \w+: Matches one or more word characters (letters, digits, and underscores).
    # re.findall() finds all matches of the pattern in the text.
  return hashtags #Returns a list of all the found hashtags.

text = "This is a #test with #special_characters_123 and #another_hashtag."
hashtags = extract_hashtags(text)
print(hashtags)

['#test', '#special_characters_123', '#another_hashtag']
